In [ ]:
# adding data augmentation 

In [86]:
import numpy as np
import copy
import os
import sys
import math
from tqdm import tqdm
from scripts import dataset
from scripts import config

from matplotlib import pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data set

In [ ]:
# load dataset
# Let's load dataset
dataset_name = 'met'

if dataset_name not in ['met', 'lsp', 'mpii']:
    raise ValueError('Unknown dataset: {}'.format(dataset_name))

if dataset_name == 'mpii':
    TEST_CV_FILEPATH = os.path.join(config.MPII_DATASET_ROOT, 'test_joints.csv')
    IMG_PATH_PREFIX = os.path.join(config.MPII_DATASET_ROOT, 'images')
    symmetric_joints = "[[12, 13], [11, 14], [10, 15], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -100500
elif dataset_name == 'lsp': # LSP dataset
    TEST_CV_FILEPATH = os.path.join(config.LSP_DATASET_ROOT, 'test_joints.csv')
    IMG_PATH_PREFIX = os.path.join(config.LSP_DATASET_ROOT, 'images')
    symmetric_joints = "[[8, 9], [7, 10], [6, 11], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -1
elif dataset_name == 'met': # MET dataset
    #TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'activity1.csv') # test only one image
    #TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'activity.csv')
    #TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'activity_with_head.csv')
    #TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'activity_wo.csv')
    #TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'activity_wo_ub.csv') # cleaned up
    TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'activity_20180618.csv') # joint rechecked! (2018.06.18) 
    #TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'activity-met_n10.csv') # joint rechecked! (2018.06.18)    
    # for debugging
    #TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'writing46.csv')  # with valid joints info
    #TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'writing46nv.csv')  # without valid joints info
    
    TEST_CV_FILEPATH = os.path.join(config.MET_DATASET_ROOT, 'activity1.csv')  # without valid joints info

    IMG_PATH_PREFIX = os.path.join(config.MET_DATASET_ROOT)
    symmetric_joints = "[[8, 9], [7, 10], [6, 11], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -1
else:
    pass
 
    
print(TEST_CV_FILEPATH)
print(IMG_PATH_PREFIX)

test_dataset = dataset.PoseDataset(
    TEST_CV_FILEPATH,
    IMG_PATH_PREFIX, 227,
    fliplr=False, rotate=False,
    shift=None,
    #bbox_extension_range=(1.0, 1.0),
    bbox_extension_range=(1.4, 1.4),
    coord_normalize=True,
    gcn=True,
    fname_index=0,
    joint_index=1,
    symmetric_joints=symmetric_joints,
    ignore_label=ignore_label,
    should_return_bbox=True,
    should_downscale_images=True,
    downscale_height=400
)

print("dataset with {} images ready".format(len(test_dataset)))

## data augmentation by horizontal flip

In [ ]:
# read one image 

from tools import augment

imageIdx = 0

image=test_dataset.get_original_image(imageIdx)    
image_id, joints = test_dataset.joints[imageIdx]
is_valid_joints, bbox = test_dataset.info[imageIdx]
print(image_id, joints, is_valid_joints, bbox)


# flip the image & joints 
image_flipped, joints_flipped, is_valid_joints_flipped, bbox_flipped = augment.flip_image_and_joints(image, joints, is_valid_joints, test_dataset.symmetric_joints, bbox)


# plot images & joints
plt.subplot(121)
plt.imshow(image[:,:,::-1])
plt.plot(joints[:,0], joints[:,1], 'or')
plt.title('original image & joints')
plt.subplot(122)
plt.imshow(image_flipped[:,:,::-1])
plt.plot(joints_flipped[:,0], joints_flipped[:,1], 'or')
plt.title('flipped image & joints')


In [ ]:
"""
Another way to plot using bbox and normalized joints

from matplotlib import pyplot as plt
%matplotlib inline

imageIdx = 0   # this image has only upper body
img=test_dataset.get_original_image(imageIdx)


image, joints, is_valid_joints, misc = test_dataset.get_example(imageIdx)
valid_joints = joints[is_valid_joints].reshape(-1,2)

#bbox=misc['orig_tightest_bbox']
bbox=misc['bbox']


# plot joints
plt.imshow(img[:,:,::-1])
# plot joints
offsetX = bbox[0]
offsetY = bbox[1]
W = bbox[2]#-bbox[0]
H = bbox[3]#-bbox[1]
print(W,H)
plt.plot((valid_joints[:,0]+0.5)*W+offsetX, (valid_joints[:,1]+0.5)*H+offsetY,'or')
"""

In [ ]:
len(test_dataset)

#image, joints, is_valid_joints, misc = test_dataset.get_example(0)

In [ ]:
# augment image 


import cv2 as cv

from tools import augment

def augmentByFlip(dataset):
    numImages = len(dataset)
    
    symmetric_joints = dataset.symmetric_joints
    

    for i in {0}:
    #for i in range (numImages):
        # get joints & info
        image_id, joints = dataset.joints[i]
        is_valid_joints, bbox = dataset.info[i]
        print(image_id, joints, is_valid_joints, bbox)
        #print(bbox)

        # get image
        image = dataset.images[image_id]

        image_flipped, joints_flipped, is_valid_joints_flipped, bbox_flipped = augment.flip_image_and_joints(image, joints, is_valid_joints, dataset.symmetric_joints, bbox)
        # below code is moved to tools/augment.py
        
        """
        H, W, C = image.shape
        print(W,H,C)

        # flip image
        image_flipped=cv.flip(image,1)

        # flip joints
        joints_flipped = joints.copy()
        joints_flipped[:,0] = W + 1 - joints_flipped[:,0]
        #print(joint_flipped)
        
        is_valid_joints_flipped = is_valid_joints.copy()
        #print(is_valid_joints_flipped)

        # swap symmetric joints!
        for i, j in symmetric_joints:
            joints_flipped[i], joints_flipped[j] = joints_flipped[j].copy(), joints_flipped[i].copy()
            is_valid_joints_flipped[i], is_valid_joints_flipped[j] = is_valid_joints_flipped[j].copy(), is_valid_joints_flipped[i].copy()
        
        print(joints_flipped)
        print(is_valid_joints_flipped)
        
        # flip bbox
        bbox_flipped = bbox.copy()
        bbox_flipped[0] = W + 1 - (bbox_flipped[0] + bbox_flipped[2])
        print(bbox_flipped)
        """
        

        # add to the dataset
        image_id_flipped = image_id+'_FLR'
        dataset.images[image_id_flipped] = image_flipped

        dataset.joints.append((image_id_flipped, joints_flipped))
        dataset.info.append((is_valid_joints, bbox_flipped))

        
        # plot!
        plt.imshow(image_flipped[:,:,::-1])
        # plot joints
        offsetX = bbox_flipped[0]
        offsetY = bbox_flipped[1]
        W = bbox_flipped[2]
        H = bbox_flipped[3]
        valid_joint_flipped = joints_flipped*is_valid_joints_flipped
        #valid_joint_flipped = joint_flipped[is_valid_joints_flipped]# .reshape(-1,2)
        
        print(valid_joint_flipped)

        plt.clf
        plt.plot(valid_joint_flipped[:,0], valid_joint_flipped[:,1],'or')  # not-normalized
        #plt.plot((valid_joint_flipped[:,0]+0.5)*W+offsetX, (valid_joint_flipped[:,1]+0.5)*H+offsetY,'or')

        
augmentByFlip(test_dataset)

        

In [ ]:
## moved to scripts/dataset.py
print(len(test_dataset))
test_dataset.augmentByFlip()
print(len(test_dataset))

## Next is to apply rotation!

In [ ]:
from tools import augment
%load_ext autoreload
%autoreload 2




imageIdx = 0

image=test_dataset.get_original_image(imageIdx)    
image_id, joints = test_dataset.joints[imageIdx]
is_valid_joints, bbox = test_dataset.info[imageIdx]
print(image_id, joints, is_valid_joints, bbox)


degree = 10
center= joints[12,:] # neck

center_hip = np.mean(joints[2:4,:],axis=0)
print(center_hip)
center_torso =  (center_hip + joints[12,:]) * 0.5
print(center_torso)



# rotate the image & joints 
#image_flipped, joints_flipped, is_valid_joints_flipped, bbox_flipped = augment.flip_image_and_joints(image, joints, is_valid_joints, test_dataset.symmetric_joints, bbox)
image_rotated, joints_rotated, is_valid_joints_rotated = augment.rotate_image_and_joints(image, joints, is_valid_joints, center_torso, degree)

# to test invalid joints
#image_rotated, joints_rotated, is_valid_joints_rotated = augment.rotate_image_and_joints(image, joints, is_valid_joints, (100,150), degree)

print(joints_rotated, is_valid_joints_rotated)


# plot images & joints
plt.subplot(121)
plt.imshow(image[:,:,::-1])
plt.plot(joints[:,0], joints[:,1], 'or')
plt.title('original image & joints')
plt.subplot(122)
plt.imshow(image_rotated[:,:,::-1])
plt.plot(joints_rotated[:,0], joints_rotated[:,1], 'or')
plt.title('rotated image & joints')



In [ ]:
## moved to scripts/dataset.py
print(len(test_dataset))
test_dataset.augmentByRotation((-10,-5,5,10))
print(len(test_dataset))

In [ ]:
for i in range(5):
    plt.subplot(1,5,i+1)
    image, joints, is_valid_joints = test_dataset.get_image_and_joints(i)  # simplified to a method!
    plt.imshow(image[:,:,::-1])
    plt.plot(joints[:,0], joints[:,1], 'or')



In [ ]:
(-10,-5,5,10)

## Next, shift

In [ ]:
# let's test the cropping function

imageIdx = 0

image=test_dataset.get_original_image(imageIdx)    

image_id, joints = test_dataset.joints[imageIdx]
is_valid_joints, bbox = test_dataset.info[imageIdx]
print(image_id, joints, is_valid_joints, bbox)


image_cropped, joints_cropped, bbox_cropped, bbox_origin = test_dataset.apply_cropping(image, joints, bbox, bbox_extension_range=None, shift=None)

print('cropped')
#print(image_id, joints, is_valid_joints, bbox)



In [ ]:

plt.subplot(121)
plt.imshow(image[:,:,::-1])

plt.subplot(122)
plt.imshow(image_cropped[:,:,::-1])
plt.plot(joints_cropped[:,0], joints_cropped[:,1],'or')


In [ ]:
print(image_id, joints, is_valid_joints, bbox)
print(joints_cropped, bbox_cropped, bbox_origin)

## augment shuffled data

In [4]:
from scripts import dataset 

def loadDataset(filename):

    dataset_name = 'met'

    TEST_CV_FILEPATH = '/var/data/MET2/' + filename
    IMG_PATH_PREFIX = '/var/data/MET2'
    symmetric_joints = "[[8, 9], [7, 10], [6, 11], [2, 3], [1, 4], [0, 5]]"
    ignore_label = -1

    print(TEST_CV_FILEPATH)
    print(IMG_PATH_PREFIX)

    test_dataset = dataset.PoseDataset(
        TEST_CV_FILEPATH,
        IMG_PATH_PREFIX, 227,
        fliplr=False, rotate=False,
        shift=None,
        #bbox_extension_range=(1.0, 1.0),
        bbox_extension_range=(1.4, 1.4),
        coord_normalize=True,
        gcn=True,
        fname_index=0,
        joint_index=1,
        symmetric_joints=symmetric_joints,
        ignore_label=ignore_label,
        should_return_bbox=True,
        should_downscale_images=True,
        downscale_height=400
    )
    
    return test_dataset

In [91]:
numClass = 10

for k in range(5):
#for k in range(1):
    train_filename = "activity-met_n{}_ub_train_k{}.csv".format(numClass, k)
    train_dataset = loadDataset(train_filename)
    #print(filename)
    
    val_filename = "activity-met_n{}_ub_val_k{}.csv".format(numClass, k)
    val_dataset = loadDataset(val_filename)
    #print(filename)
    
    print(train_filename)
    print(len(train_dataset))
    print(val_filename)
    print(len(val_dataset))

    
    # augment the training data set by rotation & horizontal flip
    train_dataset.augmentByRotation((-5,5), removeInvalidJoints=False)
    train_dataset.augmentByFlip()
    print(len(train_dataset))
    
    train_filename_augmented = "activity-met_n{}_ub_train_k{}_augmented.csv".format(numClass, k)
    train_dataset.to_csv(train_filename_augmented)
    
    val_dataset.augmentByRotation((-5,5), removeInvalidJoints=False)
    val_dataset.augmentByFlip()
    print(len(val_dataset))
    
    val_filename_augmented = "activity-met_n{}_ub_val_k{}_augmented.csv".format(numClass, k)
    val_dataset.to_csv(val_filename_augmented)
        
    
    
    
    
    
    


16it [00:00, 120.90it/s]

/var/data/MET2/activity-met_n10_ub_train_k0.csv
/var/data/MET2
Reading dataset from /var/data/MET2/activity-met_n10_ub_train_k0.csv
[1] /var/data/MET2/image/office activities/typing/45.bmp contains non-valid joints
[0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [152. 175.]
 [187. 170.]
 [194.  94.]
 [306.  82.]
 [315. 176.]
 [210. 180.]
 [230.  76.]
 [207.  12.]]
[2] /var/data/MET2/image/office activities/filing.stand/7.bmp contains non-valid joints
[0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [103. 114.]
 [129. 114.]
 [137.  74.]
 [163.  88.]
 [148. 148.]
 [ 84. 139.]
 [144.  65.]
 [136.  22.]]
[4] /var/data/MET2/image/office activities/reading.seated/15.bmp contains non-valid joints
[0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ 97. 104.]
 [ 75.  99.]
 [ 99.  64.]
 [141.  72.]
 [161.  98.]
 [146. 

70it [00:00, 204.23it/s]

Downscale image/office activities/walking about/22.png to the height 400px
[32] /var/data/MET2/image/office activities/writing/59.jpeg contains non-valid joints
[0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [113. 144.]
 [ 87. 120.]
 [114.  43.]
 [222.  73.]
 [233. 158.]
 [188. 146.]
 [166.  61.]
 [207.   8.]]
Downscale image/office activities/filing.stand/42.jpg to the height 400px
Downscale image/resting/reclining/47.jpg to the height 400px
[40] /var/data/MET2/image/office activities/typing/56.bmp contains non-valid joints
[0 0 1 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ 79. 238.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [199. 204.]
 [124. 215.]
 [100. 120.]
 [111.  90.]
 [144. 178.]
 [196. 188.]
 [137.  95.]
 [170.  49.]]
[41] /var/data/MET2/image/office activities/typing/3.bmp contains non-valid joints
[0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [1

118it [00:00, 212.79it/s]

...

Downscale image/office activities/walking about/58.png to the height 400px
[373] /var/data/MET2/image/office activities/walking about/58.png contains non-valid joints
[1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[[453. 753.]
 [443. 676.]
 [408. 454.]
 [548. 454.]
 [512. 671.]
 [ -1.  -1.]
 [365. 337.]
 [391. 361.]
 [399. 211.]
 [600. 228.]
 [583. 388.]
 [586. 501.]
 [500. 169.]
 [494.  44.]]
Downscale image/resting/reclining/31.jpg to the height 400px
[376] /var/data/MET2/image/office activities/writing/39.jpeg contains non-valid joints
[0 1 1 1 1 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ 55. 160.]
 [133. 148.]
 [197. 159.]
 [180. 169.]
 [ -1.  -1.]
 [161. 128.]
 [113. 134.]
 [128.  74.]
 [206.  85.]
 [228. 142.]
 [205. 154.]
 [164.  69.]
 [161.   9.]]
Downscale image/office activities/walking about/54.jpg to the height 400px
[378] /var/data/MET2/image/office activities/walking about/54.jpg contains non-valid joints
[0 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [264. 450.]
 [302. 294.]
 [273. 317.]
 [269. 449

92it [00:00, 269.82it/s]

[0 1 0 1 1 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ 82. 155.]
 [ -1.  -1.]
 [ 94. 161.]
 [200. 179.]
 [ -1.  -1.]
 [127.  92.]
 [137. 112.]
 [148.  72.]
 [213.  78.]
 [178. 143.]
 [121. 148.]
 [176.  61.]
 [155.  14.]]
Downscale image/office activities/walking about/53.jpg to the height 400px
Downscale image/resting/sleeping/21.jpg to the height 400px
Downscale image/office activities/filing.stand/59.PNG to the height 400px
[49] /var/data/MET2/image/office activities/filing.stand/59.PNG contains non-valid joints
[0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [275. 238.]
 [236. 331.]
 [137. 272.]
 [152. 237.]
 [240. 219.]
 [269. 161.]
 [138. 211.]
 [143. 125.]]
[50] /var/data/MET2/image/office activities/reading.seated/9.bmp contains non-valid joints
[0 0 1 1 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ 74. 123.]
 [115. 108.]
 [ -1.  -1.]
 [ -1.  -1.]
 [101.  96.]
 [ 69.  89.]
 [ 57.  51.]
 [114.  40.]
 [115.  86.]
 [136.  77.

96it [00:00, 242.55it/s]


Downscale image/office activities/walking about/22.png to the height 400px
Downscale image/office activities/reading.seated/8.jpg to the height 400px
[94] /var/data/MET2/image/office activities/reading.seated/8.jpg contains non-valid joints
[0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [266. 310.]
 [137. 360.]
 [ 99. 118.]
 [281. 135.]
 [279. 348.]
 [388. 251.]
 [222. 128.]
 [265.   5.]]
[95] /var/data/MET2/image/office activities/writing/14.bmp contains non-valid joints
[0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[[ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [ -1.  -1.]
 [141. 126.]
 [164. 140.]
 [180.  82.]
 [219.  82.]
 [221. 145.]
 [154. 154.]
 [200.  64.]
 [199.  11.]]
96 images loaded
joints shape: (14, 2)
activity-met_n10_ub_train_k4.csv
380
activity-met_n10_ub_val_k4.csv
96
[Warning] During rotation, joint [ 99. 143.] becomes invalid [ 89.01667905 143.65564237].
[Warning] During rotation, joint [  4. 261.] bec